In [16]:
import time

# Data wrangling
import pandas as pd
import numpy as np

import requests

import sqlite3

# Google API requests

from pytrends.request import TrendReq
pytrend = TrendReq(hl="en-US", tz=360)

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [17]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())

C:\Users\lvgui\AppData\Local\Temp\ipykernel_30896\910562302.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [18]:
driver.get("https://www.snopes.com/fact-check/?pagenum=1")

# Find all article titles on the page
article_list = driver.find_elements(By.CLASS_NAME, "outer_article_link_wrapper")
url_list = [article.get_attribute("href") for article in article_list]

# Print to check if output is as expected
print(f"Article list length: {len(article_list)}")
print(f"Article list length: {len(url_list)}")
print()
print("First three URLs:")
print(url_list[0:3])
print()
print("Last three URLs:")
print(url_list[-3:])

Article list length: 20
Article list length: 20

First three URLs:
['https://www.snopes.com/fact-check/did-hillary-clinton-endorse-ron-desantis-for-president/', 'https://www.snopes.com/fact-check/whole-foods-close-transgender-customers/', 'https://www.snopes.com/fact-check/tampax-hires-trans-star-dylan-mulvaney/']

Last three URLs:
['https://www.snopes.com/fact-check/is-this-sour-patch-kids-advertisment-real/', 'https://www.snopes.com/fact-check/gloria-johnson-tennessee-house-white/', 'https://www.snopes.com/fact-check/bud-light-beer-hate-gays-and-trans-people/']


In [19]:
# Iteration loop
article_list = []
url_list = []

for i in range(150):
    # Go to desired page
    driver.get(f"https://www.snopes.com/fact-check/?pagenum={i+1}")

    # Find all articles and their URLs on the page
    articles_in_page = driver.find_elements(By.CLASS_NAME, "outer_article_link_wrapper")
    article_urls_in_page = [article.get_attribute("href") for article in articles_in_page]

    # Add found articles to article_list and article_urls
    article_list = article_list + articles_in_page
    url_list = url_list + article_urls_in_page

    time.sleep(0.25)

print(f"Article list length: {len(article_list)}")
print(f"Article list length: {len(url_list)}")
print()
print("First three URLs:")
print(url_list[0:3])
print()
print("Last three titles:")
print(url_list[-3:])

Article list length: 3000
Article list length: 3000

First three URLs:
['https://www.snopes.com/fact-check/did-hillary-clinton-endorse-ron-desantis-for-president/', 'https://www.snopes.com/fact-check/whole-foods-close-transgender-customers/', 'https://www.snopes.com/fact-check/tampax-hires-trans-star-dylan-mulvaney/']

Last three titles:
['https://www.snopes.com/fact-check/neil-degrasse-tyson-vax-tweet/', 'https://www.snopes.com/fact-check/christine-grady-pfizer-vaccine/', 'https://www.snopes.com/fact-check/lyft-legal-aid-texas-abortion/']


In [20]:
def get_article_info(url, driver=driver):
    driver.get(url)

    # Get title
    try:
        title_tag = driver.find_element(By.XPATH, "//meta[@property='og:title']")
        title_content = title_tag.get_attribute("content").lower()
    except Exception:
        title_content = None

    # Get locale
    try:
        locale_tag = driver.find_element(By.XPATH, "//meta[@property='og:locale']")
        locale_content = locale_tag.get_attribute("content").lower()
    except Exception:
        locale_content = None

    # Get type
    try:
        type_tag = driver.find_element(By.XPATH, "//meta[@property='og:type']")
        type_content = type_tag.get_attribute("content").lower()
    except Exception:
        type_content = None

    # Get published_time
    try:
        pub_time_tag = driver.find_element(By.XPATH, "//meta[@property='article:published_time']")
        pub_time_content = pub_time_tag.get_attribute("content")
    except Exception:
        pub_time_content = None

    # Get modified_time
    try:
        mod_time_tag = driver.find_element(By.XPATH, "//meta[@property='article:modified_time']")
        mod_time_content = mod_time_tag.get_attribute("content")
    except Exception:
        mod_time_content = None

    # Get author
    try:
        author_tag = driver.find_element(By.XPATH, "//meta[@property='article:author']")
        author_content = author_tag.get_attribute("content").lower()
    except Exception:
        author_content = None

    # Get tag(s)
    try:
        tag_tag = driver.find_elements(By.XPATH, "//meta[@property='article:tag']")
        tag_list = [tag.get_attribute("content").lower() for tag in tag_tag]
    except Exception:
        tag_list = []

    # Get label
    try:
        label_tag = driver.find_element(By.CLASS_NAME, "rating_title_wrap")
        label_content = label_tag.text.lower().split("\n")[0]
    except Exception:
        label_content = None

    # Get article text
    try:
        body_tag = driver.find_element(By.TAG_NAME, "body")
        text_tags = body_tag.find_elements(By.TAG_NAME, "p")
        text_list = [tag.text.strip() for tag in text_tags]
        article_text = " ".join(text_list)
    except Exception:
        article_text = None

    info_dict = {
        "title": title_content,
        "locale": locale_content,
        "type": type_content,
        "published_time": pub_time_content,
        "modified_time": mod_time_content,
        "author": author_content,
        "tag_list": tag_list,
        "label": label_content,
        "article_text": article_text
    }

    return info_dict

In [21]:
article_info = {}

for url in url_list:
    article_info[url] = get_article_info(url)
    time.sleep(0.25)

In [22]:
df = (
    pd.DataFrame
    .from_dict(article_info, orient="index")
    .reset_index()
    .reset_index()
    .rename(columns={"index": "article_url", "level_0": "article_id"})
)

In [23]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2995 entries, 0 to 2994
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   article_id      2995 non-null   int64 
 1   article_url     2995 non-null   object
 2   title           2994 non-null   object
 3   locale          2994 non-null   object
 4   type            2994 non-null   object
 5   published_time  2995 non-null   object
 6   modified_time   2995 non-null   object
 7   author          2995 non-null   object
 8   tag_list        2995 non-null   object
 9   label           2974 non-null   object
 10  article_text    2957 non-null   object
dtypes: int64(1), object(10)
memory usage: 257.5+ KB
None


article_id                                        article_url  \
0           0  https://www.snopes.com/fact-check/did-hillary-...   
1           1  https://www.snopes.com/fact-check/whole-foods-...   
2           2  https://www.snopes.com/fact-check/tampax-hires...   
3           3  https://www.snopes.com/fact-check/joe-biden-20...   
4           4  https://www.snopes.com/fact-check/photo-women-...   

                                               title locale     type  \
0  did hillary clinton endorse ron desantis for p...  en_us  article   
1  no, san francisco whole foods didn't close due...  en_us  article   
2  did tampax hire trans star dylan mulvaney to p...  en_us  article   
3  yes, joe biden said he's planning to run for p...  en_us  article   
4  is this a photograph of women 'screaming throu...  en_us  article   

        published_time        modified_time                  author  \
0  2023-04-11 22:20:24  2023-04-11 22:20:24  izz scott lamagdeleine   
1  2023-04-11 18:17:59  2023-04-11 18:17:59  izz scott lamagdeleine   
2  2023-04-11 18:16:27  2023-04-11 19:34:54         damakant jayshi   
3  2023-04-11 16:15:13  2023-04-11 19:56:04         damakant jayshi   
4  2018-06-28 19:28:00  2023-04-11 15:41:57            kim lacapria   

                                            tag_list                label  \
0  [deepfakes, ron desantis, hillary clinton, ai-...                false   
1                       [whole foods, san francisco]                false   
2                                          [tampons]                false   
3            [joe biden, 2024 presidential election]  correct attribution   
4     [donald trump, uncle sam's misguided children]                false   

                                        article_text  
0   About this rating On April 11, 2023, a video ...  
1   About this rating In April 2023, Whole Foods ...  
2   About this rating On April 9, 2023, a Twitter...  
3   About this rating He followed up the remark t...  
4   About this rating On 27 June 2018 the Faceboo...

In [24]:
df_tags = df[['tag_list']]
df_tags = df_tags.explode("tag_list")
df_tags

tag_list
0                      deepfakes
0                   ron desantis
0                hillary clinton
0           ai-generated content
0     2024 presidential election
...                          ...
2992                    covid-19
2992            covid-19 vaccine
2993                    covid-19
2993            covid-19 vaccine
2994                         NaN

[6150 rows x 1 columns]

In [26]:
df.to_csv(path_or_buf="2995_snopes_articles_11042023.csv", na_rep=np.nan, index=False)

In [2]:
pytrend.build_payload(kw_list=["Donald Trump"], timeframe='today 3-m')

In [3]:
google_df_trump = pytrend.interest_over_time()
google_df_trump.head()

Donald Trump  isPartial
date                               
2023-01-08             5      False
2023-01-09             6      False
2023-01-10             5      False
2023-01-11             4      False
2023-01-12             5      False

In [ ]:
def get_google_trends_data(search_term, start_date, end_date):
    # build the payload for the pytrends API
    pytrend = TrendReq()
    pytrend.build_payload(kw_list=[search_term], timeframe=f'{start_date} {end_date}')

    # try sending the request with a delay of 1 second
    while True:
        try:
            interest_over_time_df = pytrend.interest_over_time()
            break
        except requests.exceptions.HTTPError:
            print('Too many requests, waiting...')
            time.sleep(60)

    return interest_over_time_df